<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/30_AudioSep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!git clone https://github.com/Audio-AGI/AudioSep.git
%cd AudioSep

Cloning into 'AudioSep'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 245 (delta 38), reused 37 (delta 29), pack-reused 174
Receiving objects: 100% (245/245), 16.53 MiB | 7.07 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/AudioSep


In [3]:
!pip install torchlibrosa==0.1.0 gradio==3.47.1 gdown lightning transformers==4.28.1 ftfy braceexpand webdataset soundfile wget h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!mkdir -p checkpoint
!wget "https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/audiosep_base_4M_steps.ckpt" \
    -O checkpoint/audiosep_base_4M_steps.ckpt
!wget "https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt" \
    -O checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt

--2023-12-14 10:41:22--  https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/audiosep_base_4M_steps.ckpt
Resolving huggingface.co (huggingface.co)... 54.230.18.95, 54.230.18.84, 54.230.18.110, ...
Connecting to huggingface.co (huggingface.co)|54.230.18.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/25/5f/255f11fd2743cf4de0c72f1eba5cdf82634d22af3ce4fcc5b16dbd7b7feddaca/f8cda01bfd0ebd141eef45d41db7a3ada23a56568465840d3cff04b8010ce82c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27audiosep_base_4M_steps.ckpt%3B+filename%3D%22audiosep_base_4M_steps.ckpt%22%3B&Expires=1702809682&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjgwOTY4Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yNS81Zi8yNTVmMTFmZDI3NDNjZjRkZTBjNzJmMWViYTVjZGY4MjYzNGQyMmFmM2NlNGZjYzViMTZkYmQ3YjdmZWRkYWNhL2Y4Y2RhMDFiZmQwZWJkMTQxZWVmNDVkNDFkYjdhM2F

In [7]:
import torch
from pipeline import build_audiosep, inference

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = build_audiosep(
      config_yaml='config/audiosep_base.yaml',
      checkpoint_path="checkpoint/audiosep_base_4M_steps.ckpt",
      device=device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load AudioSep model from [checkpoint/audiosep_base_4M_steps.ckpt]


In [21]:
!wget "https://audio-agi.github.io/Separate-Anything-You-Describe/demos/exp31_water drops_mixture.wav"

--2023-12-14 10:47:17--  https://audio-agi.github.io/Separate-Anything-You-Describe/demos/exp31_water%20drops_mixture.wav
Resolving audio-agi.github.io (audio-agi.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to audio-agi.github.io (audio-agi.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320044 (313K) [audio/wav]
Saving to: ‘exp31_water drops_mixture.wav’

exp31_water drops_m 100%[===================>] 312.54K  --.-KB/s    in 0.03s   

2023-12-14 10:47:17 (9.03 MB/s) - ‘exp31_water drops_mixture.wav’ saved [320044/320044]



In [22]:
audio_file = 'exp31_water drops_mixture.wav'
text = 'water drops'
output_file='separated_audio.wav'

# AudioSep processes the audio at 32 kHz sampling rate
inference(model, audio_file, text, output_file, device)

Separate audio from [exp31_water drops_mixture.wav] with textual query [water drops]
Write separated audio to [separated_audio.wav]


In [23]:
from IPython.display import Audio
Audio(output_file)

In [28]:
audio_file = "/content/drive/MyDrive/AI/AudioSep/trombone_guitar_clip.wav"
text = 'trombone melody without classic guitar'
output_file='separated_audio_trombone.wav'
inference(model, audio_file, text, output_file, device)

Separate audio from [/content/drive/MyDrive/AI/AudioSep/trombone_guitar_clip.wav] with textual query [trombone melody without classic guitar]
Write separated audio to [separated_audio_trombone.wav]


In [29]:
from IPython.display import Audio
Audio(output_file)

In [30]:
audio_file = "/content/drive/MyDrive/AI/AudioSep/trombone_guitar_clip.wav"
text = 'classic guitar'
output_file='separated_audio_guitar.wav'
inference(model, audio_file, text, output_file, device)

Separate audio from [/content/drive/MyDrive/AI/AudioSep/trombone_guitar_clip.wav] with textual query [classic guitar]
Write separated audio to [separated_audio_guitar.wav]


In [31]:
from IPython.display import Audio
Audio(output_file)